<a href="https://colab.research.google.com/github/Annie00000/Project/blob/main/1_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Q:  

若現有一份資料，有[wafer_id, y, cluster, EPI_1/EPI_2/MES_1 ] 欄位，其中

y 為各wafer_id的測量值(value，數值)。

cluster 為各wafer_id所屬的群集 (good/bad，類別型)。

EPI_1/EPI_2/MES_1 分別為wafer_id在這三種雞台下的類別資料

想要分析 哪個雞排欄位與wafer_id, y, cluster 的 相關性最大 ，亦或是說 造成這cluster狀態 是哪個機台的影響最

-----

## 一、綜合指標

**1. 綜合指標:**

$$Score = w_1 \cdot \eta^2 (\text{Mean Shift}) + w_2 \cdot \text{Cramer's V} (\text{Category Match}) + w_3 \cdot \eta^2_{var} (\text{Variance Shift})$$

(注意: 係數總和須為1，或者下方除係數總和)



**2. 指標意義**
  - $\eta^2$ (均值解釋力): 該因子各處理 vs. $y$
    - 定義： 機台 vs. $y$。衡量「欄位」對「$y$ 值中心趨勢」的解釋程度。
    - 意義： 若數值較高，代表該機台是造成 $y$ 值出現**「位移 (Mean Shift)」**的主要原因（例如：機台 A 產出值偏大，機台 B 偏小）。
  - $\eta^2_{var}$  (穩定度解釋力): 該因子各處理 vs. $y$
    - 定義： 衡量「欄位」對「$y$ 值波動劇烈程度」的解釋程度。
    - 意義： 若數值較高，代表該機台決定了生產的**「穩定性」**。這能精準捕捉到你提到的「一群穩定、一群波動」之現象，識別出哪個機台導致了製程型態的分散。
  - Cramer's V (類別一致性): 機台 vs. Cluster。
    - 定義： 該機台的類別與 Good/Bad 的分類標籤有多吻合。
    - 意義： 衡量機台編號與 Good/Bad 標籤的一致性。若數值接近 1，代表該機台的分組結果幾乎能完美預測（或對應）該 Wafer 會被歸類為哪一個 Cluster。


**3. ANOVA，為什麼選擇 $\eta^2$ (Eta-square) 而非 p-value？**

- 效應值 (Effect Size) vs. 顯著性：

  - p-value 只能告訴你「機台對 $y$ 是否有影響」，但它極度依賴樣本數。如果資料量很大，即便影響微乎其微，p-value 也會趨近於 0。它無法告訴你影響的「程度」。
  - $\eta^2$ 衡量的是 **「變異解釋比例」**。例如 $\eta^2 = 0.8$ 代表 $y$ 的總變異中有 80% 是由該機台造成的。這直接對應到你想要的「影響最深」的概念。

- 尺度統一：
  - $\eta^2$ 的範圍固定在 $0 \sim 1$ 之間。
  - Cramer's V 的範圍也是 $0 \sim 1$。
  - 兩者尺度相同，非常方便直接進行加權計算或綜合評分；而 p-value 的尺度（通常是極小值）很難直接與其他指標融合。

**4. 補充-Levene's Test:**

  - 檢定所謂的「變異數同質性」，檢驗各群之間穩定度是否有顯著差異。
  - 為何對'非正態'資料特別有效，這是因為其「變異數的檢定」轉換成「對離差（Deviations）進行 ANOVA」



---

* 備註: VBGMM 分群的，這意味著 Cluster 本身就包含了「均值（Mean）」與「變異數（Variance）」的特徵。 所以使用這套**「三合一綜合評分指標」**（Eta-square + Cramer's V + Variance Effect）非常適合

---

# **程式碼**

## 1. 基礎

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency

def analyze_machine_impact(df, setting_dict):
    """
    分析機台因子對數值偏移(Mean)、波動(Variance)及分群(Cluster)的綜合影響力
    """
    # 提取參數
    value_col = setting_dict['value_colname']
    cluster_col = setting_dict['cluster_colname']
    factors = setting_dict['factor_col_list']
    w1, w2, w3 = setting_dict['proportion']

    # 計算絕對離差 (用於衡量波動 Variance 的解釋力)
    # 使用 median 是為了對離群值更穩健 (Robust)
    y_dev = np.abs(df[value_col] - df[value_col].median())

    results = []

    for factor in factors:
        # --- 1. Eta-squared (Mean) ---
        grand_mean = df[value_col].mean()
        sst_mean = np.sum((df[value_col] - grand_mean)**2)
        group_stats = df.groupby(factor)[value_col].agg(['mean', 'count'])
        ssb_mean = np.sum(group_stats['count'] * (group_stats['mean'] - grand_mean)**2)
        eta_mean = ssb_mean / sst_mean if sst_mean != 0 else 0

        # --- 2. Eta-squared (Variance / Volatility) ---
        grand_mean_dev = y_dev.mean()
        sst_var = np.sum((y_dev - grand_mean_dev)**2)
        group_stats_dev = pd.DataFrame({'dev': y_dev, 'grp': df[factor]}).groupby('grp')['dev'].agg(['mean', 'count'])
        ssb_var = np.sum(group_stats_dev['count'] * (group_stats_dev['mean'] - grand_mean_dev)**2)
        eta_var = ssb_var / sst_var if sst_var != 0 else 0

        # --- 3. Cramer's V (Cluster Association) ---
        confusion_matrix = pd.crosstab(df[factor], df[cluster_col])
        chi2 = chi2_contingency(confusion_matrix)[0]
        n = len(df)
        phi2 = chi2 / n
        r, k = confusion_matrix.shape
        # Bias-correction
        phi2_corr = max(0, phi2 - ((k-1)*(r-1))/(n-1))
        r_corr = r - ((r-1)**2)/(n-1)
        k_corr = k - ((k-1)**2)/(n-1)

        denom = min((k_corr-1), (r_corr-1))
        cv = np.sqrt(phi2_corr / denom) if denom > 0 else 0

        # --- 綜合得分 ---
        score = (eta_mean * w1) + (eta_var * w2) + (cv * w3)

        results.append({
            'Factor': factor,
            'Eta_Mean': round(eta_mean, 4),
            'Eta_Var': round(eta_var, 4),
            'Cramers_V': round(cv, 4),
            'Score': round(score, 4)
        })

    # 建立回傳 DataFrame 並排序與排名
    result_df = pd.DataFrame(results)
    result_df = result_df.sort_values(by='Score', ascending=False).reset_index(drop=True)
    result_df['Rank'] = result_df['Score'].rank(ascending=False, method='min').astype(int)

    return result_df

# --- 使用範例 ---
# setting_dict = {
#     'value_colname': 'y',
#     'cluster_colname': 'cluster',
#     'factor_col_list': ['mes_1', 'mes_2', 'pms_1', 'pms_2'],
#     'proportion': [0.333, 0.333, 0.334]
# }
# report = analyze_machine_impact(my_df, setting_dict)
# print(report)

這份回傳清單能幫你區分出「故障的類型」：
- 如果某因子 Eta_Mean 超高： 代表該機台是造成 $y$ 值整體偏移（例如厚度變薄、阻值變大）的主因。
- 如果某因子 Eta_Var 超高： 代表該機台是造成「製程不穩定」的主因。即便平均值沒變，它讓資料變得非常散。
- 如果 Cramers_V 跟前兩者都高： 代表這台機台就是 VBGMM 能分出兩群的關鍵分水嶺。

In [ ]:
def _calculate_eta_sq(df, cat_col, num_col, sst):
    """
    加強版 Eta-squared 計算
    處理：缺失值、單一分類、SST為0等異常
    """
    # 1. 處理機台因子缺失值：將其視為一個獨立類別
    # 使用 copy() 避免 SettingWithCopyWarning
    temp_series = df[cat_col].fillna('NA')

    # 2. 檢查分類數量：若只有一個類別，無法計算變異解釋力
    unique_cats = temp_series.unique()
    if len(unique_cats) < 2:
        return 0.0

    # 3. 檢查總變異 (SST)：若 SST 為 0，代表 y 值全部相同
    if sst <= 0:
        return 0.0

    # 4. 計算 SSB (組間平方和)
    grand_mean = df[num_col].mean()
    # 這裡使用 temp_series 作為分組依據
    group_stats = df[num_col].groupby(temp_series).agg(['mean', 'count'])

    # SSB = sum( n_i * (mean_i - grand_mean)^2 )
    ssb = np.sum(group_stats['count'] * (group_stats['mean'] - grand_mean)**2)

    # 5. 計算 Eta_sq 並確保不超過 1.0 (避免浮點運算誤差)
    eta_sq = min(ssb / sst, 1.0)

    return eta_sq

In [ ]:
def _calculate_cramers_v(df, col1, col2):
    """內部函數：計算偏差修正後的 Cramer's V"""
    confusion_matrix = pd.crosstab(df[col1], df[col2])
    if confusion_matrix.empty: return 0

    chi2 = chi2_contingency(confusion_matrix)[0]
    n = len(df)
    phi2 = chi2 / n
    r, k = confusion_matrix.shape

    # Bias-correction
    phi2_corr = max(0, phi2 - ((k-1)*(r-1))/(n-1))
    r_corr = r - ((r-1)**2)/(n-1)
    k_corr = k - ((k-1)**2)/(n-1)

    denom = min((k_corr-1), (r_corr-1))
    return np.sqrt(phi2_corr / denom) if denom > 0 else 0

In [ ]:
def analyze_machine_impact(df, setting_dict):
    # 1. 參數提取
    v_col = setting_dict['value_colname']
    c_col = setting_dict['cluster_colname']
    factors = setting_dict['factor_col_list']
    w1, w2, w3 = setting_dict['proportion']

    # 2. 缺失值與資料檢查
    relevant_cols = [v_col, c_col] + factors
    missing_info = df[relevant_cols].isnull().sum()
    if missing_info.any():
        print("⚠️ 警告：偵測到缺失值，分析將暫時排除這些觀測值。")
        print(missing_info[missing_info > 0])
        df = df.dropna(subset=relevant_cols).copy()

    # 3. 預處理：計算絕對離差與全域 SST
    df['y_dev'] = np.abs(df[v_col] - df[v_col].median())

    # 預先計算 SST，避免迴圈重複運算
    sst_mean = np.sum((df[v_col] - df[v_col].mean())**2)
    sst_var = np.sum((df['y_dev'] - df['y_dev'].mean())**2)

    # 4. 迴圈計算各因子得分
    results = []
    for factor in factors:
        e_mean = _calculate_eta_sq(df, factor, v_col, sst_mean)
        e_var = _calculate_eta_sq(df, factor, 'y_dev', sst_var)
        c_v = _calculate_cramers_v(df, factor, c_col)

        score = (e_mean * w1) + (e_var * w2) + (c_v * w3)

        results.append({
            'Factor': factor,
            'Eta_Mean': e_mean,
            'Eta_Var': e_var,
            'Cramers_V': c_v,
            'Score': score
        })

    # 5. 排名與格式化
    res_df = pd.DataFrame(results).sort_values(by='Score', ascending=False).reset_index(drop=True)
    res_df['Rank'] = res_df['Score'].rank(ascending=False, method='min').astype(int)

    return res_df

## 2. 新增 hash機制

In [ ]:
import pandas as pd
import numpy as np
import hashlib
from scipy.stats import chi2_contingency

def _get_column_hash(series):
    """將欄位內容轉化為唯一的指紋 (MD5 Hash)"""
    # 處理缺失值後轉為 tuple 並進行雜湊
    # 使用 tuple 是因為它是 immutable，雜湊速度快
    # 後進行 MD5 雜湊，避免直接操作巨大的 tuple
    data_tuple = tuple(series.fillna('NA').values) # series.fillna('NA').astype(str).values.tobytes()
    return hashlib.md5(str(data_tuple).encode()).hexdigest()

def analyze_machine_impact_optimized(df, setting_dict):
    v_col = setting_dict['value_colname']
    c_col = setting_dict['cluster_colname']
    factors = setting_dict['factor_col_list']
    w1, w2, w3 = setting_dict['proportion']

    # 1. 預處理與全域 SST 計算
    df = df.copy()
    df['y_dev'] = np.abs(df[v_col] - df[v_col].median())
    sst_mean = np.sum((df[v_col] - df[v_col].mean())**2)
    sst_var = np.sum((df['y_dev'] - df['y_dev'].mean())**2)

    # 2. 快取與指紋映射字典
    cache = {}         # 儲存 {hash: {'eta_mean': x, 'eta_var': y, 'cv': z}}
    column_hashes = {} # 儲存 {column_name: hash}
    results = []

    print(f"🚀 開始分析 {len(factors)} 個欄位...")

    # 3. 執行迴圈
    for factor in factors:
        # A. 取得該欄位的內容指紋
        col_hash = _get_column_hash(df[factor])
        column_hashes[factor] = col_hash

        # B. 檢查快取 (Caching Mechanism)
        if col_hash in cache:
            stats = cache[col_hash]
            is_cached = True
        else:
            # C. 計算邏輯 (只有快取未命中時才執行)
            e_mean = _calculate_eta_sq(df, factor, v_col, sst_mean)
            e_var = _calculate_eta_sq(df, factor, 'y_dev', sst_var)
            c_v = _calculate_cramers_v(df, factor, c_col)

            stats = {
                'Eta_Mean': e_mean,
                'Eta_Var': e_var,
                'Cramers_V': c_v,
                'Unique_Cats': df[factor].fillna('NA').nunique()
            }
            cache[col_hash] = stats
            is_cached = False

        # D. 彙整結果
        score = (stats['Eta_Mean'] * w1) + (stats['Eta_Var'] * w2) + (stats['Cramers_V'] * w3)

        results.append({
            'Factor': factor,
            'Hash': col_hash,
            'Unique_Cats': stats['Unique_Cats'],
            'Eta_Mean': stats['Eta_Mean'],
            'Eta_Var': stats['Eta_Var'],
            'Cramers_V': stats['Cramers_V'],
            'Score': score,
            'Cached': is_cached  # 標註是否使用了快取
        })

    # 4. 排序與輸出
    res_df = pd.DataFrame(results).sort_values(by='Score', ascending=False).reset_index(drop=True)
    res_df['Rank'] = res_df['Score'].rank(ascending=False, method='min').astype(int)

    # 統計優化效率
    cache_hits = sum(res_df['Cached'])
    print(f"✅ 分析完成！欄位總數: {len(factors)}, 快取命中: {cache_hits} (省下 {round(cache_hits/len(factors)*100, 1)}% 計算量)")

    return res_df

## 3. multiprocessing

在使用 concurrent.futures.ProcessPoolExecutor 搭配我們之前的 結果快取 (Caching) 與 指紋辨識 (Hashing) 邏輯時，有一個核心關鍵：**「先去重，再並行」**。

如果直接把 2000 個欄位丟進多核運算，相同的欄位會被重複計算多次，造成 CPU 資源浪費。最聰明的做法是：

1. 指紋識別：在主進程先計算所有欄位的 Hash，找出「不重複的內容」。

2. 派發任務：只將「不重複的內容」派發給各個核心計算統計指標。

3. 結果映射：運算完成後，再將結果映射回原本的 2000 個欄位。

### 3-1. 版本1

In [ ]:
import pandas as pd
import numpy as np
import hashlib
from scipy.stats import chi2_contingency
from concurrent.futures import ProcessPoolExecutor
import os

# --- 核心計算子程式 (需放在 Top-level 以便 Multiprocessing 序列化) ---

def _core_compute_engine(data_dict):
    """
    在子進程中運行的計算引擎
    """
    factor_name = data_dict['factor_name']
    factor_series = data_dict['factor_series']
    y_series = data_dict['y_series']
    y_dev_series = data_dict['y_dev_series']
    cluster_series = data_dict['cluster_series']
    sst_mean = data_dict['sst_mean']
    sst_var = data_dict['sst_var']

    # 1. Eta_Mean
    e_mean = _calculate_eta_sq_logic(factor_series, y_series, sst_mean)
    # 2. Eta_Var
    e_var = _calculate_eta_sq_logic(factor_series, y_dev_series, sst_var)
    # 3. Cramer's V
    cv = _calculate_cramers_v_logic(factor_series, cluster_series)

    return {
        'fingerprint': data_dict['fingerprint'],
        'stats': {
            'e_m': e_mean,
            'e_v': e_var,
            'cv': cv,
            'u': factor_series.nunique()
        }
    }

def _calculate_eta_sq_logic(cat_series, num_series, sst):
    if sst <= 0: return 0.0
    cat_filled = cat_series.fillna('NA')
    if cat_filled.nunique() < 2: return 0.0

    grand_mean = num_series.mean()
    group_means = num_series.groupby(cat_filled).agg(['mean', 'count'])
    ssb = np.sum(group_means['count'] * (group_means['mean'] - grand_mean)**2)
    return min(ssb / sst, 1.0)

def _calculate_cramers_v_logic(cat_series, cluster_series):
    confusion_matrix = pd.crosstab(cat_series.fillna('NA'), cluster_series)
    if confusion_matrix.empty or min(confusion_matrix.shape) < 2: return 0.0

    chi2 = chi2_contingency(confusion_matrix)[0]
    n = len(cat_series)
    phi2 = chi2 / n
    r, k = confusion_matrix.shape
    phi2_corr = max(0, phi2 - ((k-1)*(r-1))/(n-1))
    r_corr = r - ((r-1)**2)/(n-1)
    k_corr = k - ((k-1)**2)/(n-1)
    denom = min((k_corr-1), (r_corr-1))
    return np.sqrt(phi2_corr / denom) if denom > 0 else 0

# --- 主程式 ---

def analyze_machine_impact_parallel(df, setting_dict):
    v_col = setting_dict['value_colname']
    c_col = setting_dict['cluster_colname']
    factors = setting_dict['factor_col_list']
    w1, w2, w3 = setting_dict['proportion']
    max_workers = setting_dict.get('max_workers', os.cpu_count())

    # 1. 預計算全域變數
    y_dev = np.abs(df[v_col] - df[v_col].median())
    sst_mean = np.sum((df[v_col] - df[v_col].mean())**2)
    sst_var = np.sum((y_dev - y_dev.mean())**2)

    # 2. 指紋辨識：找出真正需要計算的唯一欄位
    unique_tasks = {}
    factor_to_hash = {}

    for f in factors:
        # 使用 tobytes() 快速雜湊
        f_hash = hashlib.md5(df[f].fillna('NA').astype(str).values.tobytes()).hexdigest()
        factor_to_hash[f] = f_hash
        if f_hash not in unique_tasks:
            unique_tasks[f_hash] = f

    # 3. 準備並行任務清單
    task_list = []
    for f_hash, f_name in unique_tasks.items():
        task_list.append({
            'fingerprint': f_hash,
            'factor_name': f_name,
            'factor_series': df[f_name],
            'y_series': df[v_col],
            'y_dev_series': y_dev,
            'cluster_series': df[c_col],
            'sst_mean': sst_mean,
            'sst_var': sst_var
        })

    # 4. 啟動多核運算
    cache = {}
    print(f"🔥 啟動並行運算 (核心數: {max_workers})，唯一任務數: {len(task_list)}...")
    with ProcessPoolExecutor(max_workers=max_workers) as executor:
        results = list(executor.map(_core_compute_engine, task_list))

    for r in results:
        cache[r['fingerprint']] = r['stats']

    # 5. 映射回原始因子列表
    final_output = []
    for f in factors:
        f_hash = factor_to_hash[f]
        stats = cache[f_hash]
        score = (stats['e_m'] * w1) + (stats['e_v'] * w2) + (stats['cv'] * w3)
        final_output.append({
            'Factor': f,
            'Unique_Cats': stats['u'],
            'Eta_Mean': stats['e_m'],
            'Eta_Var': stats['e_v'],
            'Cramers_V': stats['cv'],
            'Score': score
        })

    return pd.DataFrame(final_output).sort_values('Score', ascending=False).reset_index(drop=True)

### 3-2. 版本2

In [ ]:
import pandas as pd
import numpy as np
import hashlib
from concurrent.futures import ProcessPoolExecutor
from scipy.stats import chi2_contingency

# --- 核心運算子程式 (必須放在頂層以便 Pickle) ---

def _worker_task(data_bundle):
    """
    單一核心執行的任務：計算 Eta_Mean, Eta_Var, Cramer's V
    data_bundle 包含: (factor_series, y_series, y_dev_series, cluster_series, sst_mean, sst_var)
    """
    f_ser, y_ser, y_dev_ser, c_ser, sst_m, sst_v = data_bundle

    # 這裡調用之前定義好的計算邏輯 (簡化版示意)
    e_mean = _calculate_eta_sq_raw(f_ser, y_ser, sst_m)
    e_var = _calculate_eta_sq_raw(f_ser, y_dev_ser, sst_v)
    cv = _calculate_cramers_v_raw(f_ser, c_ser)

    return {'e_m': e_mean, 'e_v': e_var, 'cv': cv, 'u': f_ser.nunique()}

def _calculate_eta_sq_raw(f_ser, y_ser, sst):
    if sst <= 0 or f_ser.nunique() < 2: return 0.0
    f_ser_filled = f_ser.fillna('NA')
    grand_mean = y_ser.mean()
    group_means = y_ser.groupby(f_ser_filled).mean()
    group_counts = y_ser.groupby(f_ser_filled).count()
    ssb = np.sum(group_counts * (group_means - grand_mean)**2)
    return min(ssb / sst, 1.0)

def _calculate_cramers_v_raw(f_ser, c_ser):
    confusion_matrix = pd.crosstab(f_ser.fillna('NA'), c_ser)
    if confusion_matrix.empty or min(confusion_matrix.shape) < 2: return 0.0
    chi2 = chi2_contingency(confusion_matrix)[0]
    n = len(f_ser)
    phi2 = max(0, (chi2 / n) - ((confusion_matrix.shape[1]-1)*(confusion_matrix.shape[0]-1))/(n-1))
    r_corr = confusion_matrix.shape[0] - ((confusion_matrix.shape[0]-1)**2)/(n-1)
    k_corr = confusion_matrix.shape[1] - ((confusion_matrix.shape[1]-1)**2)/(n-1)
    denom = min((k_corr-1), (r_corr-1))
    return np.sqrt(phi2 / denom) if denom > 0 else 0

# --- 主分析程式 ---

def analyze_machine_impact_multiprocess(df, setting_dict, max_workers=None):
    v_col = setting_dict['value_colname']
    c_col = setting_dict['cluster_colname']
    factors = setting_dict['factor_col_list']
    w = setting_dict['proportion']

    # 1. 預處理
    y_ser = df[v_col]
    y_dev_ser = np.abs(y_ser - y_ser.median())
    c_ser = df[c_col]
    sst_m = np.sum((y_ser - y_ser.mean())**2)
    sst_v = np.sum((y_dev_ser - y_dev_ser.mean())**2)

    # 2. 指紋辨識：找出唯一內容的欄位
    unique_patterns = {} # {hash: first_column_name}
    col_to_hash = {}    # {column_name: hash}

    for col in factors:
        # 使用 tobytes() 快速生成指紋
        h = hashlib.md5(df[col].fillna('NA').astype(str).values.tobytes()).hexdigest()
        col_to_hash[col] = h
        if h not in unique_patterns:
            unique_patterns[h] = col

    # 3. 準備並行任務 (只針對唯一的 Hash)
    unique_hashes = list(unique_patterns.keys())
    tasks = [
        (df[unique_patterns[h]], y_ser, y_dev_ser, c_ser, sst_m, sst_v)
        for h in unique_hashes
    ]

    # 4. 多核派發
    cache = {}
    print(f"🌀 原始欄位: {len(factors)}, 唯一內容: {len(unique_hashes)}。使用多核計算中...")
    with ProcessPoolExecutor(max_workers=max_workers) as executor:
        task_results = list(executor.map(_worker_task, tasks))

    for h, res in zip(unique_hashes, task_results):
        cache[h] = res

    # 5. 映射結果並計算總分
    final_results = []
    for col in factors:
        h = col_to_hash[col]
        stats = cache[h]
        score = (stats['e_m'] * w[0]) + (stats['e_v'] * w[1]) + (stats['cv'] * w[2])
        final_results.append({
            'Factor': col, 'Score': score,
            'Eta_Mean': stats['e_m'], 'Eta_Var': stats['e_v'], 'Cramers_V': stats['cv'],
            'Unique_Cats': stats['u']
        })

    return pd.DataFrame(final_results).sort_values('Score', ascending=False).reset_index(drop=True)

* **並行運算的注意事項**
1. Pickle 限制：multiprocessing 需要將資料序列化（Pickle）傳送給子進程。因此，所有子程式（如 _worker_task）必須定義在 Python 檔案的最頂層，不能定義在另一個函數內部。

2. 資料複製成本：多進程會複製一份 DataFrame 到子進程空間。如果你的 DataFrame 高達數 GB，建議只傳遞必要的 Series 而不是整個 DataFrame，如我在範例中所做的那樣。

---

## 💡**總結**

這份分析方案是專為半導體或製造業的**根因分析 (Root Cause Analysis, RCA)** 量身打造的。它結合了統計學的嚴謹性與資料工程的效能優化，旨在從成千上萬個機台欄位中，精準抓出導致產品品質偏移的「嫌疑犯」。

以下是該方案的四個核心重點總結：

---

## 1. 建立「三合一」綜合評估指標

為了同時捕捉「數值偏移」與「穩定度變化」，我們不只看平均值，而是建立了一個多維度的評分系統：

* **Eta-squared ()**：量化機台對 **y 值平均偏移**的解釋力（例如：機台 A 讓阻值集體上升）。
* **Eta-squared ()**：量化機台對 **y 值波動程度**的解釋力（例如：機台 B 讓產品品質變得很不穩定）。
* **Cramer's V**：衡量機台類別與 **VBGMM 分群標籤**的一致性（檢視機台分組是否能完美預測 Good/Bad）。

---

## 2. 針對生產資料的強健性處理

生產現場的資料往往不完美，我們在計算流程中加入防禦性機制：

* **缺失值標記**：將機台欄位的缺失（NaN）視為獨立類別 `'NA'`，因為「缺考」本身可能就是一種製程異常訊號。
* **異常過濾**：自動識別並跳過「單一分類」或「總變異為零」的無效欄位，避免程式發生除以零的錯誤。
* **預算總平方和 (SST)**：在進入迴圈前先計算全域變異，顯著提升運算效率。

---

## 3. 指紋辨識與快取技術 (Hashing & Caching)

當機台因子高達數千個時，內容重複（內容相同但名稱不同）是效能殺手：

* **指紋識別 (Hashing)**：利用 MD5 演算法將欄位內容轉化為固定長度的指紋。只要兩台機台的生產序列完全相同，指紋就會一致。
* **結果快取 (Caching)**：同指紋的機台只計算一次統計指標。若有 2000 個欄位但只有 100 種組合，計算量直接降低 **95%**。

---

## 4. 多核並行運算優化

為了壓榨硬體效能，我們採用「先去重、再並行」的策略：

* **任務派發**：只將指紋不重複的欄位派發給 `ProcessPoolExecutor`。
* **跨核加速**：多個 CPU 核心同步進行統計運算，讓處理數千個欄位的分析時間縮短至秒級。

---

### 💡 最終建議：如何解讀分析報告？

當你得到最終的 Rank 排名表時：

1. **高 Score + 高 Eta_Mean**：優先檢查機台的**參數設定**（例如溫度、壓力是否偏離中心值）。
2. **高 Score + 高 Eta_Var**：優先檢查機台的**零件老化或穩定性**（例如感測器漂移、零件磨損）。
3. **Unique_Cats 數量**：留意類別數過多的欄位（如流水號），若分數高但類別數極多，需警惕過擬合風險。

**這套工具已經準備好應對大規模的 RCA 任務。如果你需要更進一步將結果視覺化（如熱點圖或雷達圖），我也能為你提供相關代碼建議。**